# ⚙️ Configuration Setup

Before running this notebook, ensure you have:

1. **Created a `.env` file** in the project root directory
2. **Copied from `.env.example`** and filled in your actual credentials
3. **Verified your `.gitignore`** includes `.env` to protect your secrets

## Quick Setup Commands

```bash
# Copy the example file
cp .env.example .env

# Edit with your actual credentials
# (Use your preferred text editor)
```

**🔐 Security Best Practices:**
- Never commit `.env` files to version control
- Use different `.env` files for different environments (dev, staging, prod)
- Rotate your API keys regularly
- Consider using Azure Key Vault for production deployments

---

# Movies Dataset Vector Database Demo with Azure SQL Database

This notebook demonstrates how to:
1. Load the movies dataset into Azure SQL Database
2. Create tables with vector columns for storing embeddings
3. Use SQL Database as a vector database for similarity search
4. Demonstrate RAG (Retrieval Augmented Generation) patterns

## Prerequisites
- Azure SQL Database with vector support enabled
- Movies dataset files in the data/moviesdataset folder
- Required Python packages: pandas, pyodbc, numpy, openai, python-dotenv
- **Environment Configuration**: Create a `.env` file in the project root with the following variables:
  ```
  # Azure OpenAI Configuration
  AZURE_OPENAI_ENDPOINT=your_azure_openai_endpoint
  AZURE_OPENAI_API_KEY=your_azure_openai_api_key
  AZURE_OPENAI_API_VERSION=2024-06-01
  EMBEDDING_MODEL=text-embedding-ada-002
  GENERATION_MODEL=gpt-4o

  # Azure SQL Database Configuration
  SQL_SERVER=your_sql_server.database.windows.net
  SQL_DATABASE=your_database_name
  SQL_USERNAME=your_username
  SQL_PASSWORD=your_password
  ```

**⚠️ Security Note**: Never commit the `.env` file to version control. Add it to your `.gitignore` file.

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import pyodbc
import json
import ast
import warnings
from typing import List, Dict, Any
import urllib
import sqlalchemy
from sqlalchemy import create_engine, text
import time
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

# Load environment variables from .env file
load_dotenv()

# Azure OpenAI Configuration from environment variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-06-01")
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL", "text-embedding-ada-002")
GENERATION_MODEL = os.getenv("GENERATION_MODEL", "gpt-4o")

# Validate required environment variables
required_vars = ["AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
missing_vars = [var for var in required_vars if not os.getenv(var)]
if missing_vars:
    raise ValueError(f"Missing required environment variables: {', '.join(missing_vars)}")

# Initialize Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION
)

print("All libraries imported successfully!")
print(f"Azure OpenAI configured with embedding model: {EMBEDDING_MODEL}")
print(f"Generation model: {GENERATION_MODEL}")
print("✅ Environment variables loaded from .env file")

All libraries imported successfully!
Azure OpenAI configured with embedding model: text-embedding-ada-002
Generation model: gpt-4o


## 1. Load and Inspect the Movies Dataset

Let's start by loading the movies dataset and examining its structure.

In [2]:
# Load the movies metadata
movies_df = pd.read_csv(r'data\moviesdataset\movies_metadata.csv', low_memory=False)

# Load ratings data (using smaller dataset for demo)
ratings_df = pd.read_csv(r'data\moviesdataset\ratings_small.csv')

print("Movies Dataset Shape:", movies_df.shape)
print("Ratings Dataset Shape:", ratings_df.shape)
print("\nMovies Dataset Columns:")
print(movies_df.columns.tolist())
print("\nFirst few rows of movies dataset:")
movies_df.head()

Movies Dataset Shape: (45466, 24)
Ratings Dataset Shape: (100004, 4)

Movies Dataset Columns:
['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count']

First few rows of movies dataset:


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
# Clean and preprocess the data
def clean_movies_data(df):
    # Remove rows with missing essential data
    df_clean = df.dropna(subset=['title', 'overview', 'id']).copy()
    
    # Convert id to numeric, handling errors
    df_clean['id'] = pd.to_numeric(df_clean['id'], errors='coerce')
    df_clean = df_clean.dropna(subset=['id'])
    df_clean['id'] = df_clean['id'].astype(int)
    
    # Fill missing overviews
    df_clean['overview'] = df_clean['overview'].fillna('')
    
    # Create a combined text field for embedding generation
    df_clean['combined_text'] = (
        df_clean['title'].fillna('') + ' ' + 
        df_clean['overview'].fillna('') + ' ' + 
        df_clean['genres'].fillna('')
    )
    
    return df_clean

# Clean the data
movies_clean = clean_movies_data(movies_df)

# Take a subset for demo (first 1000 movies)
movies_subset = movies_clean.head(1000).copy()

print(f"Cleaned dataset shape: {movies_subset.shape}")
print(f"Sample of combined text:")
print(movies_subset['combined_text'].head(3).tolist())

Cleaned dataset shape: (1000, 25)
Sample of combined text:
["Toy Story Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences. [{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]", "Jumanji When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures. [{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]", "Grumpier Old Men A f

## 2. Connect to Azure SQL Database

Configure your connection string and establish a connection to Azure SQL Database.

In [ ]:
import os
import pyodbc

# Database connection configuration from environment variables
SERVER = os.getenv("SQL_SERVER")
DATABASE = os.getenv("SQL_DATABASE")
USERNAME = os.getenv("SQL_USERNAME")
PASSWORD = os.getenv("SQL_PASSWORD")

# Validate required database environment variables
required_db_vars = ["SQL_SERVER", "SQL_DATABASE", "SQL_USERNAME", "SQL_PASSWORD"]
missing_db_vars = [var for var in required_db_vars if not os.getenv(var)]
if missing_db_vars:
    raise ValueError(f"Missing required database environment variables: {', '.join(missing_db_vars)}")

# Connection string for pyodbc
connection_string = f"""
    Driver={{ODBC Driver 18 for SQL Server}};
    server=tcp:{SERVER};
    database={DATABASE};
    uid={USERNAME};
    pwd="{PASSWORD}";
    Encrypt=yes;
    TrustServerCertificate=no;
    Connection Timeout=30;
"""

# Alternative connection string format
connection_string = f"Driver={{ODBC Driver 17 for SQL Server}};Server=tcp:{SERVER},1433;Database={DATABASE};Uid={USERNAME};Pwd={PASSWORD};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"

def get_db_connection():
    """Get database connection using pyodbc"""
    try:
        conn = pyodbc.connect(connection_string)
        print("Successfully connected to Azure SQL Database")
        return conn
    except Exception as e:
        print(f"Error connecting to database: {e}")
        return None

# Test connection
conn = get_db_connection()
if conn:
    cursor = conn.cursor()
    cursor.execute("SELECT @@VERSION")
    version = cursor.fetchone()
    print(f"Database version: {version[0]}")
    cursor.close()
    print("✅ Database connection established using environment variables")

Successfully connected to Azure SQL Database
Database version: Microsoft SQL Azure (RTM) - 12.0.2000.8 
	May  9 2025 16:08:04 
	Copyright (C) 2025 Microsoft Corporation



## 3. Create Movies Table with Vector Column (DDL)

Create tables to store movie data and their vector embeddings.

In [ ]:
# DDL Scripts for creating tables with Azure OpenAI embeddings
def create_tables():
    """Create tables for movies and Azure OpenAI vectors"""
    
    ddl_scripts = [
        # Drop existing tables if they exist
        "DROP TABLE IF EXISTS movie_vectors;",
        "DROP TABLE IF EXISTS movies;",
        
        # Create movies table
        """
        CREATE TABLE movies (
            movie_id INT PRIMARY KEY,
            title NVARCHAR(500) NOT NULL,
            overview NVARCHAR(MAX),
            genres NVARCHAR(MAX),
            release_date DATE,
            budget BIGINT,
            revenue BIGINT,
            runtime FLOAT,
            vote_average FLOAT,
            vote_count INT,
            popularity FLOAT,
            original_language NVARCHAR(10),
            combined_text NVARCHAR(MAX),
            created_at DATETIME2 DEFAULT GETDATE()
        );
        """,
        
        # Create movie vectors table with Azure OpenAI embedding dimensions
        """
        CREATE TABLE movie_vectors (
            movie_id INT PRIMARY KEY,
            title NVARCHAR(500) NOT NULL,
            embedding VECTOR(1536),  -- 1536-dimensional vector for Azure OpenAI text-embedding-3-small
            embedding_model NVARCHAR(100) DEFAULT 'text-embedding-3-small',
            created_at DATETIME2 DEFAULT GETDATE(),
            FOREIGN KEY (movie_id) REFERENCES movies(movie_id)
        );
        """,
        
        # Create index for vector similarity search
        """
        CREATE INDEX idx_movie 
        ON movie_vectors(movie_id);
        """
    ]
    
    if conn:
        cursor = conn.cursor()
        for script in ddl_scripts:
            try:
                print(f"Executing: {script[:50]}...")
                cursor.execute(script)
                conn.commit()
                print("✅ Success")
            except Exception as e:
                print(f"Error: {e}")
                conn.rollback()
        cursor.close()
        print("All tables created successfully for Azure OpenAI embeddings!")
    else:
        print("No database connection")

# Create the tables
create_tables()

Executing: DROP TABLE IF EXISTS movie_vectors;...
✅ Success
Executing: DROP TABLE IF EXISTS movies;...
✅ Success
Executing: 
        CREATE TABLE movies (
            movie_i...
✅ Success
Executing: 
        CREATE TABLE movie_vectors (
            ...
✅ Success
Executing: 
        CREATE INDEX idx_movie 
        ON movie_...
✅ Success
✅ All tables created successfully for Azure OpenAI embeddings!


## 4. Generate Azure OpenAI Vector Embeddings

Create vector embeddings for movies using Azure OpenAI text-embedding-3-small model.

In [26]:
# Generate vector embeddings using Azure OpenAI
def get_azure_openai_embedding(text, model=EMBEDDING_MODEL):
    """Get embedding from Azure OpenAI"""
    try:
        response = client.embeddings.create(
            input=text,
            model=model
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Error getting embedding: {e}")
        return None

def generate_movie_embeddings_batch(movies_df, batch_size=100):
    """Generate Azure OpenAI embeddings for movies in batches"""
    
    embeddings = []
    failed_count = 0
    
    print(f"Generating embeddings for {len(movies_df)} movies using {EMBEDDING_MODEL}...")
    print(f"Processing in batches of {batch_size}")
    
    for i in range(0, len(movies_df), batch_size):
        batch = movies_df.iloc[i:i+batch_size]
        print(f"Processing batch {i//batch_size + 1}/{(len(movies_df)-1)//batch_size + 1}")
        
        batch_texts = batch['combined_text'].tolist()
        
        try:
            # Get embeddings for the entire batch
            response = client.embeddings.create(
                input=batch_texts,
                model=EMBEDDING_MODEL
            )
            
            # Extract embeddings from response
            batch_embeddings = [item.embedding for item in response.data]
            embeddings.extend(batch_embeddings)
            
            # Rate limiting - wait between batches
            time.sleep(1)
            
        except Exception as e:
            print(f"Error processing batch {i//batch_size + 1}: {e}")
            # Add None for failed embeddings
            embeddings.extend([None] * len(batch_texts))
            failed_count += len(batch_texts)
            
            # Wait longer on error
            time.sleep(5)
    
    print(f"Embedding generation completed!")
    print(f"Successfully generated: {len(embeddings) - failed_count} embeddings")
    print(f"Failed: {failed_count} embeddings")
    
    return embeddings

# Generate embeddings - using smaller subset for demo to manage API costs
print("Note: Using first 50 movies for demo to manage Azure OpenAI API costs")
movies_demo = movies_subset.head(50).copy()

# Generate embeddings
embeddings = generate_movie_embeddings_batch(movies_demo, batch_size=10)

# Add embeddings to dataframe, handling failed cases
movies_demo['embedding'] = embeddings
movies_demo = movies_demo.dropna(subset=['embedding'])  # Remove rows with failed embeddings

print(f"Final dataset size: {len(movies_demo)} movies with embeddings")
print(f"Embedding dimensions: {len(movies_demo['embedding'].iloc[0]) if len(movies_demo) > 0 else 'N/A'}")

if len(movies_demo) > 0:
    print("Sample embedding (first 10 dimensions):")
    print(movies_demo['embedding'].iloc[0][:10])

Note: Using first 50 movies for demo to manage Azure OpenAI API costs
Generating embeddings for 50 movies using text-embedding-ada-002...
Processing in batches of 10
Processing batch 1/5
Processing batch 2/5
Processing batch 3/5
Processing batch 4/5
Processing batch 5/5
Embedding generation completed!
Successfully generated: 50 embeddings
Failed: 0 embeddings
Final dataset size: 50 movies with embeddings
Embedding dimensions: 1536
Sample embedding (first 10 dimensions):
[-0.012333696708083153, -0.04674243927001953, -0.01232033409178257, -0.02140691503882408, -0.01333589293062687, -0.008692382834851742, 0.015179933980107307, -0.004546630661934614, -0.01769210584461689, -0.02005729079246521]


## 5. Insert Movie Data and Vectors into SQL Database

Load the movie data and their vector embeddings into the database.

In [ ]:
# Insert movie data with Azure OpenAI embeddings into database
def insert_movie_data(movies_df):
    """Insert movie data and Azure OpenAI vectors into database"""
    
    if not conn:
        print("No database connection")
        return
    
    cursor = conn.cursor()
    
    try:
        print("Inserting movie data with Azure OpenAI embeddings...")
        
        movies_inserted = 0
        vectors_inserted = 0
        
        for idx, row in movies_df.iterrows():
            try:
                # Parse and clean data
                movie_id = int(row['id'])
                title = str(row['title'])[:500]  # Truncate if too long
                overview = str(row['overview']) if pd.notna(row['overview']) else None
                genres = str(row['genres']) if pd.notna(row['genres']) else None
                
                # Parse release_date
                release_date = None
                if pd.notna(row['release_date']):
                    try:
                        release_date = pd.to_datetime(row['release_date']).date()
                    except:
                        pass
                
                # Parse numeric fields
                budget = int(row['budget']) if pd.notna(row['budget']) and str(row['budget']).isdigit() else None
                revenue = int(row['revenue']) if pd.notna(row['revenue']) and str(row['revenue']).isdigit() else None
                runtime = float(row['runtime']) if pd.notna(row['runtime']) else None
                vote_average = float(row['vote_average']) if pd.notna(row['vote_average']) else None
                vote_count = int(row['vote_count']) if pd.notna(row['vote_count']) else None
                popularity = float(row['popularity']) if pd.notna(row['popularity']) else None
                original_language = str(row['original_language'])[:10] if pd.notna(row['original_language']) else None
                combined_text = str(row['combined_text'])
                
                # Insert into movies table
                cursor.execute("""
                    INSERT INTO movies (movie_id, title, overview, genres, release_date, budget, 
                                      revenue, runtime, vote_average, vote_count, popularity, 
                                      original_language, combined_text)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                """, (movie_id, title, overview, genres, release_date, budget, revenue, 
                     runtime, vote_average, vote_count, popularity, original_language, combined_text))
                
                movies_inserted += 1
                
                # Insert Azure OpenAI embedding into movie_vectors table
                if pd.notna(row['embedding']) and row['embedding'] is not None:
                    embedding_str = '[' + ','.join(map(str, row['embedding'])) + ']'
                    cursor.execute("""
                        INSERT INTO movie_vectors (movie_id, title, embedding, embedding_model)
                        VALUES (?, ?, CAST(? AS VECTOR(1536)), 'text-embedding-3-small')
                    """, (movie_id, title, embedding_str))
                    
                    vectors_inserted += 1
                
                if movies_inserted % 10 == 0:
                    print(f"Inserted {movies_inserted} movies, {vectors_inserted} vectors...")
                    conn.commit()
                    
            except Exception as e:
                print(f"Error inserting movie {row.get('id', 'unknown')}: {e}")
                continue
        
        # Final commit
        conn.commit()
        print(f"✅ Successfully inserted {movies_inserted} movies and {vectors_inserted} Azure OpenAI vectors")
        
    except Exception as e:
        print(f"❌ Error during insertion: {e}")
        conn.rollback()
    finally:
        cursor.close()

# Insert the data (only movies with successful embeddings)
insert_movie_data(movies_demo)

Inserting movie data with Azure OpenAI embeddings...
Error inserting movie 862: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error inserting movie 8844: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error inserting movie 15602: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error inserting movie 31357: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error inserting movie 11862: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error inserting movie 949: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error inserting movie 11860: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
Error inserting movie 45325: The truth value of an array with more than one element is ambiguous. Use a.a

## 6. Query Movies Table from Python

Verify the data insertion and run basic queries.

In [ ]:
# Query the database to verify data insertion
def run_basic_queries():
    """Run basic queries to verify data"""
    
    if not conn:
        print("❌ No database connection")
        return
    
    cursor = conn.cursor()
    
    # Check row counts
    cursor.execute("SELECT COUNT(*) FROM movies")
    movies_count = cursor.fetchone()[0]
    print(f"Total movies in database: {movies_count}")
    
    cursor.execute("SELECT COUNT(*) FROM movie_vectors")
    vectors_count = cursor.fetchone()[0]
    print(f"Total vectors in database: {vectors_count}")
    
    # Show sample movie data
    print("\n📽️ Sample movies:")
    cursor.execute("""
        SELECT TOP 5 movie_id, title, vote_average, popularity 
        FROM movies 
        ORDER BY popularity DESC
    """)
    
    results = cursor.fetchall()
    for row in results:
        print(f"ID: {row[0]}, Title: {row[1]}, Rating: {row[2]}, Popularity: {row[3]}")
    
    # Show genres distribution
    print("\n🎭 Sample movie with genres:")
    cursor.execute("""
        SELECT TOP 3 title, overview, genres 
        FROM movies 
        WHERE genres IS NOT NULL AND LEN(genres) > 10
    """)
    
    for row in cursor.fetchall():
        print(f"Title: {row[0]}")
        print(f"Overview: {row[1][:100]}...")
        print(f"Genres: {row[2]}")
        print("-" * 50)
    
    cursor.close()

# Run basic queries
run_basic_queries()

## 7. Calculate Vector Similarity in SQL Database

Demonstrate how to perform vector similarity calculations directly in SQL Database.

In [ ]:
# SQL Vector Similarity Functions Demo
def demonstrate_vector_similarity():
    """Demonstrate vector similarity calculations in SQL"""
    
    if not conn:
        print(" No database connection")
        return
    
    cursor = conn.cursor()
    
    print("Vector Similarity Demonstrations\n")
    
    # 1. Basic vector distance calculation between two specific movies
    print("1️ Calculate similarity between two specific movies:")
    cursor.execute("""
        DECLARE @movie1_vector VECTOR(100)
        DECLARE @movie2_vector VECTOR(100)
        
        SELECT @movie1_vector = embedding FROM movie_vectors WHERE title LIKE '%Toy Story%'
        SELECT @movie2_vector = embedding FROM movie_vectors WHERE title LIKE '%Jumanji%'
        
        SELECT 
            1 - VECTOR_DISTANCE('cosine', @movie1_vector, @movie2_vector) AS cosine_similarity,
            VECTOR_DISTANCE('euclidean', @movie1_vector, @movie2_vector) AS euclidean_distance
    """)

## 8. Advanced Vector Search with Python Integration

Implement semantic search functionality by combining Python text processing with SQL vector operations.

In [ ]:
# Advanced semantic search functionality using Azure OpenAI
def semantic_movie_search_azure_openai(query_text, top_k=5):
    """Perform semantic search for movies using Azure OpenAI embeddings"""
    
    if not conn:
        print("No database connection")
        return []
    
    # Get query embedding from Azure OpenAI
    print(f"Getting embedding for query: '{query_text}'")
    query_embedding = get_azure_openai_embedding(query_text)
    
    if query_embedding is None:
        print("Failed to get query embedding")
        return []
    
    # Convert to SQL vector format
    query_vector_str = '[' + ','.join(map(str, query_embedding)) + ']'
    
    cursor = conn.cursor()
    
    try:
        # Find most similar movies using Azure OpenAI embeddings
        cursor.execute(f"""
            SELECT TOP {top_k}
                mv.title,
                m.overview,
                m.genres,
                m.vote_average,
                m.popularity,
                1 - VECTOR_DISTANCE('cosine', mv.embedding, CAST(? AS VECTOR(1536))) AS similarity_score
            FROM movie_vectors mv
            JOIN movies m ON mv.movie_id = m.movie_id
            ORDER BY similarity_score DESC
        """, (query_vector_str,))
        
        results = cursor.fetchall()
        cursor.close()
        return results
        
    except Exception as e:
        print(f"Error during search: {e}")
        cursor.close()
        return []

# Example searches using Azure OpenAI embeddings
print("🔍 Semantic Movie Search with Azure OpenAI Embeddings\\n")

# Search 1: Action movies
print("1️⃣ Search: 'action adventure superhero movies with explosions'")
results = semantic_movie_search_azure_openai("action adventure superhero movies with explosions", top_k=3)
for i, row in enumerate(results, 1):
    print(f"  {i}. {row[0]} (Score: {row[5]:.4f})")
    print(f"     Overview: {row[1][:100] if row[1] else 'N/A'}...")
    print(f"     Rating: {row[3]}, Popularity: {row[4]:.1f}")
    print()

print("-" * 60)

# Search 2: Romance movies
print("2️⃣ Search: 'romantic love story with emotional drama'")
results = semantic_movie_search_azure_openai("romantic love story with emotional drama", top_k=3)
for i, row in enumerate(results, 1):
    print(f"  {i}. {row[0]} (Score: {row[5]:.4f})")
    print(f"     Overview: {row[1][:100] if row[1] else 'N/A'}...")
    print()

print("-" * 60)

# Search 3: Family movies
print("3️⃣ Search: 'family friendly animated movie for children'")
results = semantic_movie_search_azure_openai("family friendly animated movie for children", top_k=3)
for i, row in enumerate(results, 1):
    print(f"  {i}. {row[0]} (Score: {row[5]:.4f})")
    print(f"     Overview: {row[1][:100] if row[1] else 'N/A'}...")
    print()

## 9. RAG with Azure OpenAI GPT-4o Mini

Implement a complete RAG (Retrieval Augmented Generation) system using SQL Database as vector store and Azure OpenAI GPT-4o Mini for generation.

In [ ]:
# RAG Implementation using Azure OpenAI GPT-4o Mini and SQL Database as Vector Store

def get_azure_openai_completion(messages, model=GENERATION_MODEL, max_tokens=1000):
    """Get completion from Azure OpenAI GPT-4o Mini"""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            temperature=0.7
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error getting completion: {e}")
        return None

def rag_movie_recommendation(user_query, top_k=5):
    """Complete RAG pipeline for movie recommendations"""
    
    print(f"🎬 **RAG Movie Recommendation System**")
    print(f"Query: '{user_query}'\\n")
    
    # Step 1: Retrieve relevant movies using vector similarity
    print("🔍 Step 1: Retrieving relevant movies...")
    retrieved_movies = semantic_movie_search_azure_openai(user_query, top_k=top_k)
    
    if not retrieved_movies:
        return "Sorry, I couldn't find any relevant movies for your query."
    
    # Step 2: Format context for GPT-4o Mini
    print("📝 Step 2: Formatting context for GPT-4o Mini...")
    
    context_parts = []
    for i, movie in enumerate(retrieved_movies, 1):
        title, overview, genres, rating, popularity, similarity = movie
        context_parts.append(
            f"Movie {i}: {title}\\n"
            f"Overview: {overview or 'No overview available'}\\n"
            f"Genres: {genres or 'Unknown'}\\n"
            f"Rating: {rating}/10, Popularity: {popularity:.1f}\\n"
            f"Similarity Score: {similarity:.4f}\\n"
        )
    
    context = "\\n".join(context_parts)
    
    # Step 3: Generate response using GPT-4o Mini
    print("🤖 Step 3: Generating recommendation with GPT-4o Mini...")
    
    system_prompt = """You are a knowledgeable movie recommendation assistant. Based on the provided movie data from a vector database search, give personalized movie recommendations that match the user's query. 
    
    Instructions:
    - Analyze the retrieved movies and their similarity scores
    - Recommend the most relevant movies based on the user's preferences
    - Explain why each movie matches their query
    - Include brief details about plot, genre, and ratings
    - Be conversational and helpful
    - If applicable, suggest similar themes or related movies"""
    
    user_prompt = f"""User Query: {user_query}
    
    Retrieved Movies from Vector Database:
    {context}
    
    Please provide personalized movie recommendations based on this data."""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    recommendation = get_azure_openai_completion(messages)
    
    return {
        "query": user_query,
        "retrieved_movies": retrieved_movies,
        "recommendation": recommendation,
        "context_used": context
    }

# Example RAG queries
print("🎯 **RAG Demo with Azure OpenAI GPT-4o Mini**\\n")

# RAG Example 1
query1 = "I want to watch a thrilling action movie with great special effects"
result1 = rag_movie_recommendation(query1)

if result1:
    print("\\n" + "="*80)
    print("🎬 **RAG RECOMMENDATION 1**")
    print("="*80)
    print(f"**Query:** {result1['query']}")
    print(f"\\n**GPT-4o Mini Recommendation:**")
    print(result1['recommendation'])
    print("\\n" + "="*80)

# RAG Example 2
query2 = "Recommend me a heartwarming family movie for weekend viewing"
result2 = rag_movie_recommendation(query2)

if result2:
    print("\\n🎬 **RAG RECOMMENDATION 2**")
    print("="*80)
    print(f"**Query:** {result2['query']}")
    print(f"\\n**GPT-4o Mini Recommendation:**")
    print(result2['recommendation'])
    print("\\n" + "="*80)

print("\\n✅ **RAG System Components:**")
print("• **Embedding Model:** text-embedding-3-small (1536 dimensions)")
print("• **Vector Database:** Azure SQL Database with VECTOR data type")
print("• **Generation Model:** GPT-4o Mini")
print("• **Similarity Function:** Cosine similarity")
print("• **Retrieval Strategy:** Top-K similarity search")

## 10. Performance Analysis and SQL Database Vector Features

Analyze the performance of vector operations and explore SQL Database vector capabilities.

In [ ]:
# Performance Analysis and Azure OpenAI Vector Features Exploration\nimport time\n\ndef analyze_azure_openai_vector_performance():\n    \"\"\"Analyze Azure OpenAI vector search performance\"\"\"\n    \n    if not conn:\n        print(\"❌ No database connection\")\n        return\n    \n    cursor = conn.cursor()\n    \n    print(\"⚡ **Azure OpenAI Vector Performance Analysis**\\n\")\n    \n    # 1. Test query performance with 1536-dimensional vectors\n    print(\"1️⃣ **Vector Similarity Query Performance (1536D):**\")\n    \n    start_time = time.time()\n    cursor.execute(\"\"\"\n        DECLARE @target_vector VECTOR(1536)\n        SELECT @target_vector = embedding FROM movie_vectors WHERE movie_id = (SELECT MIN(movie_id) FROM movie_vectors)\n        \n        SELECT TOP 10\n            title,\n            1 - VECTOR_DISTANCE('cosine', embedding, @target_vector) AS similarity\n        FROM movie_vectors\n        ORDER BY similarity DESC\n    \"\"\")\n    \n    results = cursor.fetchall()\n    end_time = time.time()\n    \n    print(f\"   • Query executed in: {(end_time - start_time)*1000:.2f} ms\")\n    print(f\"   • Results returned: {len(results)}\")\n    if results:\n        print(f\"   • Top result: {results[0][0]} (similarity: {results[0][1]:.4f})\")\n    \n    # 2. Check Azure OpenAI embedding storage efficiency\n    print(\"\\n2️⃣ **Azure OpenAI Embedding Storage:**\")\n    cursor.execute(\"\"\"\n        SELECT \n            COUNT(*) as total_vectors,\n            embedding_model,\n            AVG(CAST(LEN(CAST(embedding AS NVARCHAR(MAX))) AS FLOAT)) as avg_vector_size_chars\n        FROM movie_vectors\n        GROUP BY embedding_model\n    \"\"\")\n    \n    storage_stats = cursor.fetchall()\n    for stat in storage_stats:\n        print(f\"   • Model: {stat[1]}\")\n        print(f\"   • Total vectors: {stat[0]}\")\n        print(f\"   • Average size: {stat[2]:.0f} characters\")\n    \n    # 3. Vector distance metrics comparison with high-dimensional vectors\n    print(\"\\n3️⃣ **Distance Metrics with 1536D Vectors:**\")\n    cursor.execute(\"\"\"\n        DECLARE @vec1 VECTOR(1536) = (SELECT TOP 1 embedding FROM movie_vectors ORDER BY movie_id)\n        DECLARE @vec2 VECTOR(1536) = (SELECT TOP 1 embedding FROM movie_vectors ORDER BY movie_id DESC)\n        \n        SELECT \n            'Cosine Distance' as metric,\n            VECTOR_DISTANCE('cosine', @vec1, @vec2) as distance,\n            1 - VECTOR_DISTANCE('cosine', @vec1, @vec2) as similarity\n        UNION ALL\n        SELECT \n            'Euclidean Distance' as metric,\n            VECTOR_DISTANCE('euclidean', @vec1, @vec2) as distance,\n            NULL as similarity\n    \"\"\")\n    \n    metrics = cursor.fetchall()\n    for metric in metrics:\n        if metric[2] is not None:\n            print(f\"   • {metric[0]}: {metric[1]:.4f} (Similarity: {metric[2]:.4f})\")\n        else:\n            print(f\"   • {metric[0]}: {metric[1]:.4f}\")\n    \n    # 4. Index effectiveness analysis\n    print(\"\\n4️⃣ **Index Information:**\")\n    cursor.execute(\"\"\"\n        SELECT \n            i.name as index_name,\n            i.type_desc,\n            s.name as table_name\n        FROM sys.indexes i\n        JOIN sys.tables s ON i.object_id = s.object_id\n        WHERE s.name = 'movie_vectors'\n    \"\"\")\n    \n    indexes = cursor.fetchall()\n    for idx in indexes:\n        print(f\"   • Index: {idx[0]}, Type: {idx[1]}, Table: {idx[2]}\")\n    \n    cursor.close()\n\n# Run performance analysis\nanalyze_azure_openai_vector_performance()\n\nprint(\"\\n\" + \"=\"*80)\nprint(\"\\n🎯 **Azure OpenAI Integration Summary:**\")\nprint(\"\\n✅ **Key Capabilities Demonstrated:**\")\nprint(\"   • Native 1536-dimensional vector support in Azure SQL Database\")\nprint(\"   • Azure OpenAI text-embedding-3-small integration\")\nprint(\"   • GPT-4o Mini for RAG generation\")\nprint(\"   • Cosine similarity search with high-dimensional vectors\")\nprint(\"   • Real-time semantic search capabilities\")\n\nprint(\"\\n🚀 **Production Use Cases:**\")\nprint(\"   • Semantic movie search and recommendations\")\nprint(\"   • Content-based filtering systems\")\nprint(\"   • RAG applications for customer support\")\nprint(\"   • Multi-modal search combining text and metadata\")\nprint(\"   • Personalized content discovery\")\n\nprint(\"\\n💰 **Cost Optimization Tips:**\")\nprint(\"   • Cache embeddings to avoid repeated API calls\")\nprint(\"   • Use batch processing for embedding generation\")\nprint(\"   • Implement embedding versioning for updates\")\nprint(\"   • Monitor token usage and implement rate limiting\")\nprint(\"   • Consider embedding reuse for similar content\")\n\nprint(\"\\n📈 **Performance Best Practices:**\")\nprint(\"   • Create appropriate indexes on vector columns\")\nprint(\"   • Use TOP N queries to limit result sets\")\nprint(\"   • Consider partitioning for very large datasets\")\nprint(\"   • Monitor query execution plans\")\nprint(\"   • Implement connection pooling for high-throughput scenarios\")\n\nprint(\"\\n🔧 **Azure OpenAI Configuration:**\")\nprint(f\"   • Embedding Model: {EMBEDDING_MODEL} (1536 dimensions)\")\nprint(f\"   • Generation Model: {GENERATION_MODEL}\")\nprint(f\"   • API Version: {AZURE_OPENAI_API_VERSION}\")\nprint(\"   • Rate limiting and retry logic implemented\")\nprint(\"   • Secure API key management recommended\")

## 11. Cleanup and Next Steps

### Cleanup Resources

In [ ]:
# Cleanup resources (uncomment to run)
def cleanup_resources():
    """Clean up database resources"""
    
    if not conn:
        print(" No database connection")
        return
    
    cursor = conn.cursor()
    
    cleanup_scripts = [
        "DROP TABLE IF EXISTS movie_vectors;",
        "DROP TABLE IF EXISTS movies;"
    ]
    
    print("🧹 Cleaning up database resources...")
    
    for script in cleanup_scripts:
        try:
            cursor.execute(script)
            print(f"✅ Executed: {script}")
        except Exception as e:
            print(f"❌ Error: {e}")
    
    conn.commit()
    cursor.close()
    print("✅ Cleanup completed")

# Uncomment the next line to clean up resources
# cleanup_resources()

# Close database connection
if conn:
    conn.close()
    print("📝 Database connection closed")

print("\\n🎬 **Azure OpenAI Vector Database Demo Complete!** 🎬")
print("\\nYou have successfully demonstrated:")
print("• Loading movie dataset into Azure SQL Database")
print("• Creating tables with 1536-dimensional vector columns")
print("• Generating Azure OpenAI embeddings using text-embedding-3-small")
print("• Storing and indexing high-dimensional vectors")
print("• Performing semantic search with Azure OpenAI embeddings")
print("• Implementing RAG with GPT-4o Mini and SQL Database vector store")
print("• Optimizing performance for production workloads")

print("\\n🔧 **Next Steps for Production:**")
print("• Set up proper Azure OpenAI API key management")
print("• Implement comprehensive error handling and retry logic")
print("• Add monitoring and logging for API usage")
print("• Scale vector dimensions based on your specific use case")
print("• Implement caching strategies to optimize costs")
print("• Consider Azure OpenAI content filtering policies")

print("\\n💡 **Advanced Patterns to Explore:**")
print("• Hybrid search combining vector and keyword search")
print("• Multi-modal embeddings for images and text")
print("• Vector database partitioning for massive scale")
print("• Real-time embedding updates and versioning")
print("• Custom fine-tuned embedding models")